In [112]:
import pandas as pd

In [113]:
df = pd.read_csv("../raw_data/ar_properties.csv")

In [114]:
df.columns

Index(['id', 'ad_type', 'start_date', 'end_date', 'created_on', 'lat', 'lon',
       'l1', 'l2', 'l3', 'l4', 'l5', 'l6', 'rooms', 'bedrooms', 'bathrooms',
       'surface_total', 'surface_covered', 'price', 'currency', 'price_period',
       'title', 'description', 'property_type', 'operation_type'],
      dtype='object')

In [115]:
df = df.rename(columns={
    "lat":"latitude",
    "lon":"longitude",
    "l1":"country", 
    "l2":"province", 
    "l3":"city",
    "l4":"district",
    "l5":"estate"
})

In [116]:
df.columns

Index(['id', 'ad_type', 'start_date', 'end_date', 'created_on', 'latitude',
       'longitude', 'country', 'province', 'city', 'district', 'estate', 'l6',
       'rooms', 'bedrooms', 'bathrooms', 'surface_total', 'surface_covered',
       'price', 'currency', 'price_period', 'title', 'description',
       'property_type', 'operation_type'],
      dtype='object')

In [117]:
df.l6.value_counts()

Series([], Name: count, dtype: int64)

In [118]:
df = df.drop(columns=["l6"])

In [119]:
df.ad_type.value_counts()

ad_type
Propiedad    1000000
Name: count, dtype: int64

In [120]:
df = df.drop(columns=["ad_type"])

In [121]:
df.operation_type.value_counts()

operation_type
Venta                782122
Alquiler             183927
Alquiler temporal     33951
Name: count, dtype: int64

In [122]:
df["operation_type"] = df["operation_type"].replace(regex={
    "Venta":"Sale",
    "Alquiler temporal":"Temporary Rent",
    "Alquiler":"Rent"})

In [123]:
df.operation_type.value_counts()

operation_type
Sale              782122
Rent              183927
Temporary Rent     33951
Name: count, dtype: int64

In [124]:
df.property_type.value_counts()

property_type
Departamento       373376
Otro               239720
Casa               190023
Lote                83792
PH                  35217
Local comercial     34493
Oficina             26365
Cochera              8908
Depósito             6502
Casa de campo        1604
Name: count, dtype: int64

In [125]:
df["property_type"] = df["property_type"].replace(regex={
    "Departamento":"Apartment",
    "Local comercial":"Commercial Premises",
    "Oficina":"Office",
    "Depósito":"Warehouse",
    "Otro": "Other",
    "Casa de campo":"Village House",
    "Casa":"House",
    "Cochera":"Garage",
    "Lote":"Terrain",
    "PH": "??????"
    })

In [127]:
cond_rent = df["operation_type"] == "Rent"
cond_sale = df["operation_type"] == "Sale"
cond_curr = df["currency"] == "USD"
df[cond_sale & cond_curr].price.mean(), \
    df[cond_rent & cond_curr].price.mean()

(231140.34107096842, 7379.891555198346)